>Curso: Curso Superior de Tecnologia em Banco de Dados: Ênfase em Data Analytics<br>
>Disciplina: Programação para Dados<br>
>Projeto: Fase 2<br>
>Aluno: Rafael de Menezes Ehlers<br>
>Data: Setembro de 2024

### Importa as bibliotecas necessárias

In [244]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Classe para carregar os dados

In [245]:
class DataLoader:
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)

    def get_data(self):
        return self.data

### Classe para pré-processamento

In [246]:
class PreProcessing:
    def __init__(self, data):
        self.data = data

    def clean_data(self):
        # Tratar valores nulos, converter tipos, etc.
        self.data['Release date'] = pd.to_datetime(self.data['Release date'], errors='coerce')
        self.data = self.data.dropna(subset=['Metacritic score'])
        return self.data

### Funções para responder às perguntas

In [247]:
class Analysis:
    def __init__(self, data):
        self.data = data

    # Pergunta 1: Jogos mais bem avaliados
    def top_metacritic_games_grouped_by_name(self, top_n=10):
        # Agrupa os jogos por nome e mantém o maior Metacritic score por nome de jogo
        grouped_games = self.data.loc[self.data.groupby('Name')['Metacritic score'].idxmax()]
        
        # Ordena os jogos por Metacritic score (decrescente) e, em caso de empate, pela data de lançamento (crescente)
        sorted_games = grouped_games.sort_values(
            by=['Metacritic score', 'Release date'],
            ascending=[False, True]
        )
        
        # Retorna os top N jogos
        return sorted_games[['Name', 'Release date', 'Metacritic score']].head(top_n)

    # Pergunta 2: Estatísticas para jogos de Role-playing
    def role_playing_stats(self):
        # Filtra os jogos do gênero "Role-playing" usando .loc
        role_playing_games = self.data.loc[self.data['Genres'].str.contains('RPG', na=False)].copy()

        # Verifica se existem jogos de role-playing
        if role_playing_games.empty:
            return {
                'DLC mean': 0,
                'DLC max': 0,
                'Positive mean': 0,
                'Positive max': 0,
                'Negative mean': 0,
                'Negative max': 0,
                'Demo materials mean': 0,
                'Demo materials max': 0
            }

        # Converte as colunas relevantes para numéricas ou conta os elementos, tratando valores ausentes
        role_playing_games['DLC count'] = pd.to_numeric(role_playing_games['DLC count'], errors='coerce').fillna(0)
        role_playing_games['Positive'] = pd.to_numeric(role_playing_games['Positive'], errors='coerce').fillna(0)
        role_playing_games['Negative'] = pd.to_numeric(role_playing_games['Negative'], errors='coerce').fillna(0)

        # Conta o número de screenshots e filmes usando str.split() e len(), e preenche NaN com 0
        role_playing_games['Screenshots'] = role_playing_games['Screenshots'].fillna('').apply(lambda x: len(x.split(',')) if x else 0)
        role_playing_games['Movies'] = role_playing_games['Movies'].fillna('').apply(lambda x: len(x.split(',')) if x else 0)

        # Calcula o número total de materiais de demonstração (Screenshots + Movies)
        role_playing_games = role_playing_games.assign(
            Demo_materials=role_playing_games['Screenshots'] + role_playing_games['Movies']
        )

        # Calcula estatísticas: média e máximo de DLCs, avaliações positivas, avaliações negativas e materiais de demonstração
        stats = {
            'DLCs médio': round(role_playing_games['DLC count'].mean(), 2),
            'DLCs máximo': round(role_playing_games['DLC count'].max(), 2),
            'Avaliações Positivas média': round(role_playing_games['Positive'].mean(), 2),
            'Avaliações Positivas  máximo': round(role_playing_games['Positive'].max(), 2),
            'Avaliações Negativas média': round(role_playing_games['Negative'].mean(), 2),
            'Avaliações Negativas máximo': round(role_playing_games['Negative'].max(), 2),
            'Materias de demonstração média': round(role_playing_games['Demo_materials'].mean(), 2),
            'Materias de demonstração máximo': round(role_playing_games['Demo_materials'].max(), 2)
        }

        return stats

    # Pergunta 3: Empresas que mais publicam jogos pagos
    # Função para encontrar os cinco publishers que mais publicam jogos pagos e calcular as estatísticas de avaliações positivas
    def top_publishers(self):
        # Filtra apenas jogos pagos
        paid_games = self.data[self.data['Price'] > 0]

        # Contabiliza o número de jogos pagos por publisher
        publisher_counts = paid_games['Publishers'].value_counts().head(5)

        # Lista para armazenar as estatísticas de cada publisher
        publisher_stats = []

        # Itera sobre os publishers mais frequentes
        for publisher in publisher_counts.index:
            # Filtra os jogos pagos do publisher
            publisher_games = paid_games[paid_games['Publishers'] == publisher]

            # Calcula a média e a mediana de avaliações positivas
            positive_mean = publisher_games['Positive'].mean()
            positive_median = publisher_games['Positive'].median()

            # Adiciona os resultados à lista
            publisher_stats.append({
                'Produtora': publisher,
                'Jogos Pagos': publisher_counts[publisher],
                'Média Rev. Pos.': round(positive_mean, 2),
                'Mediana Rev. Pos.': round(positive_median, 2)
            })

        # Cria um DataFrame a partir dos resultados
        df = pd.DataFrame(publisher_stats)

        return df

    # Pergunta 4: Crescimento de suporte para Linux
    def linux_support_growth(self):
        linux_support = self.data[self.data['Linux'] == True]
        linux_games_by_year = linux_support.groupby(linux_support['Release date'].dt.year).size()
        return linux_games_by_year

### Funções para criar gráficos


In [248]:
class Graphs:
    def __init__(self, data):
        self.data = data

    # Gráfico 1: Percentual de jogos com suporte para cada sistema operacional
    def os_support_pie_chart(self):
        os_support = self.data[['Windows', 'Mac', 'Linux']].sum()
        os_support.plot.pie(autopct='%1.1f%%')
        plt.title('Percentual de Jogos com Suporte para Sistemas Operacionais')
        plt.show()

    # Gráfico 2: Jogos single-player Indie e Estratégia por ano
    def indie_strategy_trend(self):
        indie_games = self.data[self.data['Genres'].str.contains('Indie', na=False)]
        strategy_games = self.data[self.data['Genres'].str.contains('Strategy', na=False)]
        indie_by_year = indie_games.groupby(indie_games['Release date'].dt.year).size()
        strategy_by_year = strategy_games.groupby(strategy_games['Release date'].dt.year).size()

        plt.plot(indie_by_year.index, indie_by_year.values, label='Indie', marker='o')
        plt.plot(strategy_by_year.index, strategy_by_year.values, label='Strategy', marker='x')
        plt.title('Número de Jogos Indie e Estratégia Single-Player Lançados por Ano (2010-2020)')
        plt.xlabel('Ano')
        plt.ylabel('Número de Jogos')
        plt.legend()
        plt.show()

### Execução do programa

1. Carregar o Dataset:

In [249]:
loader = DataLoader('steam_games.csv')
data = loader.get_data()

2. Pré-processar o Dataset:



In [250]:
pre_processor = PreProcessing(data)
clean_data = pre_processor.clean_data()

3. Realizar as análises:



In [251]:
analysis = Analysis(clean_data)


4. Respondendo as perguntas:

>Pergunta 1: Quais são os dez jogos mais bem avaliados, de acordo com o Metacritic? No caso de notas repetidas, ordenar os jogos de acordo com suas datas de lançamento (do mais velho para o mais recente).  



In [252]:
top_games = analysis.top_metacritic_games_grouped_by_name(10)
#print(top_games)


>Pergunta 2: Para jogos de role-playing, qual o número médio e máximo de: DLCs, avaliações positivas, avaliações negativas e materiais de demonstração (número de capturas de tela e filmes, somados)?  

In [253]:
rpg_stats = analysis.role_playing_stats()
# Formata a saída no formato "Label: Valor" com quebra de linha
rpg_stats = "\n".join([f"{label}: {valor}" for label, valor in rpg_stats.items()])
#print(rpg_stats)

>Pergunta 3: Quais são as cinco empresas que mais publicam jogos pagos na plataforma? Para tais empresas, qual o número médio e mediano de avaliações positivas de seus jogos pagos?  



In [254]:
top_publishers = analysis.top_publishers()
print(top_publishers)

        Produtora  Jogos Pagos  Média Rev. Pos.  Mediana Rev. Pos.
0  Big Fish Games          443             7.36                5.0
1          8floor          239             4.46                3.0
2  Strategy First          162           276.45               23.0
3    Laush Studio          157            19.78               12.0
4        HH-Games          156            10.54                9.0


4. Gerar Gráficos:



In [255]:
#graphs = Graphs(clean_data)
#graphs.os_support_pie_chart()